<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://assets.vespa.ai/logos/Vespa-logo-green-RGB.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg">
  <img alt="#Vespa" width="200" src="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg" style="margin-bottom: 25px;">
</picture>

# Creating a code search application with Vespa and ModernBERT

In this notebook, we will demonstrate how to build a code search application using [Vespa](https://vespa.ai/) and the recently released [Alibaba-NLP/gte-modernbert-base](https://huggingface.co/Alibaba-NLP/gte-modernbert-base).

In december 2024, [Answer.AI](www.answer.ai) and [LightOn](https://www.lighton.ai/) recently [announced](https://huggingface.co/blog/modernbert) ModernBERT, a new and modernized version of [BERT](https://huggingface.co/papers/1810.04805).

BERT was released in 2018, and is in 2025 still _the_ most downloaded text model from [Huggingface Hub](https://huggingface.co/models?sort=downloads) (and 3rd overall). There have been many learnings and improvements since then, and ModernBERT's goal was to incorporate all the learnings from the past 7 years when training a new base model.


![ModernBERT](https://www.answer.ai/posts/2024-12-19-modernbert/modernbert_pareto_curve.png)

And as you can see in the image above, they succeeded. ModernBERT is a pareto improvement (both speed and performance) over BERT and its descendants. 

Like BERT, ModernBERT is a base model, which is trained on the masked language model (MLM) objective. (Predict a masked token). This means that it is a general-purpose model that can (and should) be fine-tuned on a wide range of tasks, like text classification, named entity recognition, and retrieval.

And only a month after its release, we are already starting to see many fine-tuned models based on ModernBERT.

Some notable examples are:
- [jxm/cde-small-v2](https://huggingface.co/jxm/cde-small-v2)
- [Alibaba-NLP/gte-modernbert-base](https://huggingface.co/Alibaba-NLP/gte-modernbert-base)

To view all the models that are fine-tuned on ModernBERT, you can check out its model tree at [Huggingface Hub](https://huggingface.co/models?other=base_model:finetune:answerdotai/ModernBERT-base).

## Code retrieval is where ModernBERT shines

As explained in this insightful [blog post](https://jina.ai/news/what-should-we-learn-from-modernbert/) by Jina.ai, ModernBERT performs particularly well on code retrieval, largely because is trained on a lot of code, and uses the [OLMo-tokenizer](https://huggingface.co/docs/transformers/en/model_doc/olmo), whic also was trained on code.

![code tokens](https://jina-ai-gmbh.ghost.io/content/images/2025/01/code_tokens-cheat-2.svg)

As we can see, the ModernBERT tokenizer does a better job of tokenizing code.

The ModernBERT authors shared their interest for programming-related tasks in their [blog post](https://huggingface.co/blog/modernbert):

> ... out of all the existing open source encoders, ModernBERT is in a class of its own on programming-related tasks. We’re particularly interested in what downstream uses this will lead to, in terms of improving programming assistants.

## Code retrieval benchmark

[CoIR](https://arxiv.org/pdf/2407.02883) is a comprehensive benchmark for code information retrieval. 

![CoIR](../_static/coir.png)


<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>

In [36]:
#!pip3 install pyvespa vespacli datasets

## Retrieving the ONNX model for use with Vespa

Vespa supports both [embedding](https://docs.vespa.ai/en/embedding.html#huggingface-embedder) and [ranking](https://docs.vespa.ai/en/onnx.html) with ONNX models.

Previously, we would have to convert each model to ONNX format for use with Vespa.

Lucky for us, [Tom Aarsen](https://huggingface.co/tomaarsen) and [transformers.js](https://github.com/huggingface/transformers.js)-maintainer [@xenova](https://huggingface.co/Xenova) have done an amazing job in making sure new huggingface models are converted to ONNX format for us.


## Find the model you are interested in using with Vespa on Huggingface

https://huggingface.co/models?pipeline_tag=sentence-similarity&library=onnx&sort=trending is a good starting point.

When you have found a model you are interested in using, click the "Files"-tab and locate the download button of the ONNX model-file, right-click and copy the download link.

![Download ONNX model](../_static/download_hf.png)

## Making sure the configuration is correct

The [Vespa docs](https://docs.vespa.ai/en/reference/embedding-reference.html#huggingface-embedder) specifies 

## Huggingface Embedder

An embedder using any Huggingface tokenizer, including multilingual tokenizers, to produce tokens which is then input to a supplied transformer model in ONNX model format.

The Huggingface embedder is configured in services.xml, within the `container` tag:

```
<container id="default" version="1.0">
    <component id="hf-embedder" type="hugging-face-embedder">
        <transformer-model path="my-models/model.onnx"/>
        <tokenizer-model path="my-models/tokenizer.json"/>
        <prepend>
          <query>query:</query>
          <document>passage:</document>
        </prepend>
    </component>
    ...
</container>
```

### Huggingface embedder reference config

In addition to embedder ONNX parameters:

Name | Occurrence | Description | Type | Default  
---|---|---|---|---  
transformer-model | One | Use to point to the transformer ONNX model file | model-type | N/A  
tokenizer-model | One | Use to point to the `tokenizer.json` Huggingface tokenizer configuration file | model-type | N/A  
max-tokens | One | The maximum number of tokens accepted by the transformer model | numeric | 512  
transformer-input-ids | One | The name or identifier for the transformer input IDs | string | input_ids  
transformer-attention-mask | One | The name or identifier for the transformer attention mask | string | attention_mask  
transformer-token-type-ids | One | The name or identifier for the transformer token type IDs. If the model does not use `token_type_ids` use `<transformer-token-type-ids/>` | string | token_type_ids  
transformer-output | One | The name or identifier for the transformer output | string | last_hidden_state  
pooling-strategy | One | How the output vectors of the ONNX model is pooled to obtain a single vector representation. Valid values are `mean` and `cls` | string | mean  
normalize | One | A boolean indicating whether to normalize the output embedding vector to unit length (length 1). Useful for `prenormalized-angular` distance-metric | boolean | false  
prepend | Optional | Prepend instructions that are prepended to the text input before tokenization and inference. Useful for models that have been trained with specific prompt instructions. The instructions are prepended to the input text. | Element `<query>` and/or `<document>`  | Optional query/doc prepend instruction.


In [20]:
model_url = "https://huggingface.co/Alibaba-NLP/gte-modernbert-base/resolve/main/onnx/model_quantized.onnx?download=true"

tokenizer_url = "https://huggingface.co/Alibaba-NLP/gte-modernbert-base/resolve/main/tokenizer.json?download=true"

We will also run inference on a sample query to verify that we get the same results in Vespa as in the original model.

In [8]:
from datasets import load_dataset

ds = load_dataset("CoIR-Retrieval/cosqa", "corpus")

In [10]:
docs = ds["corpus"].take(1000)

In [11]:
len(docs)

1000

In [12]:
sample_text, sample_id = docs[0]["text"], docs[0]["_id"]

In [13]:
from IPython.display import display, Markdown


def display_snippet(snippet):
    display(Markdown(f"**Snippet:**\n\n```python\n{snippet}\n```"))


display_snippet(sample_text)

**Snippet:**

```python
def writeBoolean(self, n):
        """
        Writes a Boolean to the stream.
        """
        t = TYPE_BOOL_TRUE

        if n is False:
            t = TYPE_BOOL_FALSE

        self.stream.write(t)
```

In [18]:
model_name = "gte-modernbert-base"
model_dims = 768

## Create an application package

The [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage)
has all the Vespa configuration files -
create one from scratch:

In [53]:
from vespa.package import ServicesConfiguration
from vespa.configuration.services import (
    services,
    container,
    search,
    document_api,
    document_processing,
    content,
    redundancy,
    documents,
    document,
    node,
    nodes,
    component,
    components,
    transformer_model,
    tokenizer_model,
)
from vespa.configuration.vt import vt

application_name = "codesearch"
schema_name = "snippets"


services_config = ServicesConfiguration(
    application_name=application_name,
    services_config=services(
        container(
            search(),
            document_api(),
            document_processing(),
            components(
                component(
                    transformer_model(url=model_url),
                    tokenizer_model(url=tokenizer_url),
                    vt("transformer-token-type-ids"),
                    vt("max_tokens", 8192),
                    type="hugging-face-embedder",
                    id=model_name,
                ),
            ),
            id=f"{application_name}_container",
            version="1.0",
        ),
        content(
            redundancy("1"),
            nodes(node(distribution_key="0", hostalias="node1")),
            documents(document(type=schema_name, mode="index")),
            id=f"{application_name}_content",
            version="1.0",
        ),
    ),
)

In [54]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    HNSW,
    RankProfile,
    FieldSet,
    GlobalPhaseRanking,
    Function,
)


package = ApplicationPackage(
    name=application_name,
    schema=[
        Schema(
            name=schema_name,
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(
                        name="text",
                        type="string",
                        indexing=["index", "summary"],
                        index="enable-bm25",
                    ),
                    Field(
                        name="embedding",
                        type=f"tensor<float>(x[{model_dims}])",
                        indexing=[
                            "input text",
                            f"embed {model_name}",
                            "summary",
                            "index",
                            "attribute",
                        ],
                        ann=HNSW(distance_metric="angular"),
                        is_document_field=False,
                    ),
                ]
            ),
            fieldsets=[
                FieldSet(
                    name="default",
                    fields=[
                        "text",
                    ],
                )
            ],
            rank_profiles=[
                RankProfile(
                    name="bm25",
                    inputs=[("query(q)", f"tensor<float>(x[{model_dims}])")],
                    functions=[Function(name="bm25sum", expression="bm25(text)")],
                    first_phase="bm25sum",
                ),
                RankProfile(
                    name="semantic",
                    inputs=[("query(q)", f"tensor<float>(x[{model_dims}])")],
                    first_phase="closeness(field, embedding)",
                ),
                RankProfile(
                    name="fusion",
                    inherits="bm25",
                    inputs=[("query(q)", f"tensor<float>(x[{model_dims}])")],
                    first_phase="closeness(field, embedding)",
                    global_phase=GlobalPhaseRanking(
                        expression="reciprocal_rank_fusion(bm25sum, closeness(field, embedding))",
                        rerank_count=10,
                    ),
                ),
            ],
        )
    ],
    services_config=services_config,
    # components=[
    #     Component(
    #         id=f"{model_name}",
    #         type="hugging-face-embedder",
    #         parameters=[
    #             Parameter(
    #                 "transformer-model",
    #                 {"url": model_url.split("?")[0]},
    #             ),
    #             Parameter(
    #                 "tokenizer-model",
    #                 {"url": tokenizer_url.split("?")[0]},
    #             ),
    #             Parameter(
    #                 "transformer-token-type-ids",
    #                 {},
    #             ),
    #             Parameter(
    #                 "max-tokens",
    #                 {},
    #                 "8192",  # The ModernBERT model has a maximum input length of 8192 tokens, but it is memory-intensive
    #             ),
    #         ],
    #     )
    # ],
)

Note that the name cannot have `-` or `_`.

## Deploy the Vespa application 

Deploy `package` on the local machine using Docker,
without leaving the notebook, by creating an instance of
[VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaDocker). `VespaDocker` connects
to the local Docker daemon socket and starts the [Vespa docker image](https://hub.docker.com/r/vespaengine/vespa/). 

If this step fails, please check
that the Docker daemon is running, and that the Docker daemon socket can be used by clients (Configurable under advanced settings in Docker Desktop).

In [55]:
package.to_files(application_name)

In [56]:
from vespa.deployment import VespaCloud
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Replace with your tenant name from the Vespa Cloud Console
tenant_name = "vespa-team"

vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=application_name,
    key_content=os.getenv(
        "VESPA_TEAM_API_KEY", None
    ),  # Key is only used for CI/CD testing of this notebook. Can be removed if logging in interactively
    application_package=package,
)

Setting application...
Running: vespa config set application vespa-team.codesearch
Setting target cloud...
Running: vespa config set target cloud

Api-key found for control plane access. Using api-key.


In [57]:
app = vespa_cloud.deploy()

Deployment started in run 3 of dev-aws-us-east-1c for vespa-team.codesearch. This may take a few minutes the first time.
INFO    [14:48:22]  Deploying platform version 8.466.24 and application dev build 3 for dev-aws-us-east-1c of default ...
INFO    [14:48:22]  Using CA signed certificate version 1
INFO    [14:48:23]  Using 1 nodes in container cluster 'codesearch_container'
INFO    [14:48:24]  Using 1 nodes in container cluster 'codesearch_container'
INFO    [14:48:27]  Session 331814 for tenant 'vespa-team' prepared and activated.
INFO    [14:48:27]  ######## Details for all nodes ########
INFO    [14:48:28]  h97538f.dev.us-east-1c.aws.vespa-cloud.net: expected to be UP
INFO    [14:48:28]  --- platform vespa/cloud-tenant-rhel8:8.466.24
INFO    [14:48:28]  --- container-clustercontroller on port 19050 has config generation 331805, wanted is 331814
INFO    [14:48:28]  --- metricsproxy-container on port 19092 has config generation 331805, wanted is 331814
INFO    [14:48:28]  h107467b.d

`app` now holds a reference to a [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) instance.

## Feeding documents to Vespa

TODO

In [30]:
from datasets import load_dataset

vespa_feed = [
    {"id": str(i), "fields": {"text": doc["text"], "id": doc["_id"]}}
    for i, doc in enumerate(docs)
]

Now we can feed to Vespa using `feed_iterable` which accepts any `Iterable` and an optional callback function where we can
check the outcome of each operation. The application is configured to use [embedding](https://docs.vespa.ai/en/embedding.html)
functionality, that produce a vector embedding using a concatenation of the title and the body input fields. This step is computionally expensive. Read more
about embedding inference in Vespa in the [Accelerating Transformer-based Embedding Retrieval with Vespa](https://blog.vespa.ai/accelerating-transformer-based-embedding-retrieval-with-vespa/).

In [31]:
import nest_asyncio

nest_asyncio.apply()

In [32]:
from vespa.io import VespaResponse, VespaQueryResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(f"Error when feeding document {id}: {response.get_json()}")


app.feed_async_iterable(vespa_feed, schema=schema_name, callback=callback)

## Querying Vespa


### Plain Keyword search 
The following uses plain keyword search functionality with [bm25](https://docs.vespa.ai/en/reference/bm25.html) ranking, the `bm25` rank-profile was configured in the 
application package to use a linear combination of the bm25 score of the query terms against the title and the body field. 


In [33]:
with app.syncio(connections=1) as session:
    query = "python compare timespan to number"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding,q))",
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()

In [34]:
response.json

{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 1683},
  'coverage': {'coverage': 100,
   'documents': 11001,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'children': [{'id': 'id:snippets:snippets::8',
    'relevance': 0.03252247488101534,
    'source': 'codesearch_content',
    'fields': {'sddocname': 'snippets',
     'documentid': 'id:snippets:snippets::8',
     'embedding': {'type': 'tensor<float>(x[768])',
      'values': [0.008496503345668316,
       -0.5039947628974915,
       0.6639868021011353,
       1.212415337562561,
       -0.05594464763998985,
       0.8520852327346802,
       -0.7542403936386108,
       0.5785810947418213,
       -0.6646630764007568,
       1.1322063207626343,
       -2.018076181411743,
       -2.048189878463745,
       0.7192966938018799,
       -0.9589497447013855,
       1.2892142534255981,
       -0.3528820276260376,
       -0.6725801229476929,
       -0.056216850876808167,
       0.62251293659210

In [35]:
display_snippet(response.hits[0]["fields"]["text"])

**Snippet:**

```python
def timespan(start_time):
    """Return time in milliseconds from start_time"""

    timespan = datetime.datetime.now() - start_time
    timespan_ms = timespan.total_seconds() * 1000
    return timespan_ms
```

## Cleanup

In [24]:
vespa_cloud.delete()

Deactivated vespa-team.codesearch in dev.aws-us-east-1c
Deleted instance vespa-team.codesearch.default


## Next steps

This is just an intro into the capabilities of Vespa and pyvespa.
Browse the site to learn more about schemas, feeding and queries - 
find more complex applications in
[examples](https://pyvespa.readthedocs.io/en/latest/examples.html).